In [2]:

from transformers import PegasusForConditionalGeneration
# Need to download tokenizers_pegasus.py and other Python script from Fengshenbang-LM github repo in advance,
# or you can download tokenizers_pegasus.py and data_utils.py in https://huggingface.co/IDEA-CCNL/Randeng_Pegasus_523M/tree/main
# Strongly recommend you git clone the Fengshenbang-LM repo:
# 1. git clone https://github.com/IDEA-CCNL/Fengshenbang-LM
# 2. cd Fengshenbang-LM/fengshen/examples/pegasus/
# and then you will see the tokenizers_pegasus.py and data_utils.py which are needed by pegasus model
from tokenizers_pegasus import PegasusTokenizer

import torch


mps = torch.device("mps")

model = PegasusForConditionalGeneration.from_pretrained("IDEA-CCNL/Randeng-Pegasus-523M-Summary-Chinese", device_map=mps)
tokenizer = PegasusTokenizer.from_pretrained("IDEA-CCNL/Randeng-Pegasus-523M-Summary-Chinese")



# model Output: 反垄断调查小组突击查访奔驰上海办事处，对多名奔驰高管进行约谈


In [6]:
text = "Raft使用心跳机制触发leader election。服务器启动后都处在follower状态。如果follower能收到leader或candidate的消息，它就会维持在follower状态。\
Leader会定时发送心跳消息给所有follower。如果follower经过一定时间没有收到消息，它就会将自身转化为candidate，启动election（增加当前term number，启动新term。注意，其它follower可能用同样的方式加入这个term，成为term的candidate）\
Candidate会投票给自己，然后通过RPC向集群的其它所有服务器发送RequestVote RPC。投票可能有3种结果：\
Candidate自身获选（超过半数票）\
另一个Candidate获选\
超时，没有获选者\
每个服务器在一个term只能投票给一个candidate，采用先到先得的方式（candidate会投自己），这样，超半数票的要求就限制了一次election只可能选出一个leader。一旦选出新leader，新leader就开始向其它服务器发送心跳，防止新election发生\
在等待选举的时候，candidate可能收到AppendEntries RPC，发送方可能是之前断开的leader，也可能是收到多数票的新leader。如果发送方的term number和该candidate一样或更大（更大的原因是选举timeout，又启动了新term），如果小于自身的term number，则candidate拒绝请求，继续等待选举结果\
如果同时有多个follower成为candidate，选票可能分散，没有candidate能得到超半数票。Raft会指定一个election timeout，超过这个时间candidate会继续增加term number，启动新选举。新发出的RequestVote会带有更大的term number\
为避免选举得不到结果的情况反复出现，Raft使用一个固定区间内的随机election timeout（好比在一个平均通讯时间10ms的网络里，election timeout在150ms-300ms之间）。这样，大部分情况下只有一个candidate会timeout，并迅速启动/完成新一轮election。\
Raft最初考虑采用排榜的方式处理多个candidate分选票的情况。亦即给每个服务器分配一个不同的排名，如果一个candidate收到更高排名的RequestVote，它会放弃candidate身份，回到follower。但是这种方式在某些情况下变得很复杂。Raft最终选择随机超时重试的方式，因为这种方式更易于理解"
inputs = tokenizer(text, max_length=1024, return_tensors="pt")

# Generate Summary
summary_ids = model.generate(inputs["input_ids"])
tokenizer.batch_decode(summary_ids, skip_special_tokens=True, clean_up_tokenization_spaces=False)

['raft 使用心跳机制触发leader election']

: 